In [ ]:
import pandas as pd
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
model  = 'gemma3:27b' # 'mistral-small3.1' #'gemma3:27b'
def promptModel(prompt):
    class Guess(BaseModel):
        guess: str
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.guess
print(promptModel('What is 1+2*3+5?  Return a single number.')) # force the model to load

In [ ]:
def createPrompt(clue1, clue1_len, clue1_offset, clue2, clue2_len, clue2_offset):
    _prompt_ = '''You are a crossword puzzle solver. Two clues are given below. Each clue corresponds to a word, and the two words intersect at a common character. For each clue, the following information is provided:

Clue – A brief hint or definition.

Answer length – The number of letters in the answer word.

Index of intersection – The index (starting from 0) of the letter in this word that intersects with the other word.

Your task is to find the most likely pair of words that match the clues and intersect at the given positions with the same letter. Prioritize common crossword-style answers and typical word usage.

Input Format:

Clue 1: "<clue text>", Length: <int>, Intersection Index: <int>  
Clue 2: "<clue text>", Length: <int>, Intersection Index: <int>

OutputFormat:

{
  "answer1": "<answer1>",
  "answer2": "<answer2>",
  "letter": "<common letter>",
   "index_in_answer1": <int>,
   "index_in_answer2": <int>
}

Example Input:

Clue 1: "Feline pet", Length: 3, Intersection Index: 1  
Clue 2: "To catch or seize", Length: 4, Intersection Index: 2

Example Output:

{
  "answer1": "CAT",
  "answer2": "GRAB",
  "letter": "A",
  "index_in_answer1": 1,
  "index_in_answer2": 2
}

Input:

Clue 1: "[__CLUE1__]", Length: [__CLUE1_LEN__], Intersection Index: [__CLUE1_I__]
Clue 2: "[__CLUE2__]", Length: [__CLUE2_LEN__], Intersection Index: [__CLUE2_I__]

Output:
'''
    _prompt_ = _prompt_.replace('[__CLUE1__]', clue1)
    _prompt_ = _prompt_.replace('[__CLUE1_LEN__]', str(clue1_len))
    _prompt_ = _prompt_.replace('[__CLUE1_I__]', str(clue1_offset))
    _prompt_ = _prompt_.replace('[__CLUE2__]', clue2)
    _prompt_ = _prompt_.replace('[__CLUE2_LEN__]', str(clue2_len))
    _prompt_ = _prompt_.replace('[__CLUE2_I__]', str(clue2_offset))
    return _prompt_

def promptModel(prompt):
    class Guess(BaseModel):
        answer1: str
        answer2: str
        letter: str
        index_in_answer1: int
        index_in_answer2: int
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.answer1.replace(' ',''), guess.answer2.replace(' ','')

_cross_file_    = _dir_ + '20250501_xwords_cross.parquet'
_per_cell_file_ = _dir_ + '20250501_xwords_orientations.parquet'

down_lu, across_lu = {}, {}
_answers_  = {}
def saveResults():
    _for_df_ = {'clue_num': [], 'orientation': [], 'answer': [], 'count': []}
    for _tuple_ in _answers_.keys():
        for _answer_ in _answers_[_tuple_].keys():
            _cluenum_, _orientation_ = _tuple_
            _count_ = _answers_[_tuple_][_answer_]
            _for_df_['clue_num'].append(_cluenum_)
            _for_df_['orientation'].append(_orientation_)
            _for_df_['answer'].append(_answer_)
            _for_df_['count'].append(_count_)
    df_cross = pd.DataFrame(_for_df_)
    #df_cross.to_parquet(_cross_file_)
    _for_df_ = {'xi':[], 'yi':[], 'orientation':[], 'answer':[]}
    for _tuple_ in down_lu.keys():
        _for_df_['xi'].append(_tuple_[0])
        _for_df_['yi'].append(_tuple_[1])
        _for_df_['orientation'].append('down')
        _for_df_['answer'].append(down_lu[_tuple_])
    for _tuple_ in across_lu.keys():
        _for_df_['xi'].append(_tuple_[0])
        _for_df_['yi'].append(_tuple_[1])
        _for_df_['orientation'].append('across')
        _for_df_['answer'].append(across_lu[_tuple_])
    df_lu = pd.DataFrame(_for_df_)
    #df_lu.to_parquet(_per_cell_file_)

def updateAnswers(_tuple_, _answer_):
    if _tuple_  not in _answers_.keys():          _answers_[_tuple_]           = {}
    if _answer_ not in _answers_[_tuple_].keys(): _answers_[_tuple_][_answer_] = 0
    _answers_[_tuple_][_answer_] += 1
    if len(_answers_)%10 == 0: saveResults()

def cleanupAnswer(s):
    s, r = s.lower(), ''
    for i in range(len(s)):
        if s[i] >= 'a' and s[i] <= 'z': r += s[i]
    return r

if os.path.exists(_cross_file_) == False and os.path.exists(_per_cell_file_) == False:
    for yi in range(xwords.y_tiles):
        for xi in range(xwords.x_tiles):
            _cell_ = xwords.cells[yi][xi]
            if _cell_.isBlocker() == False:
                dy = 0
                while yi+dy > 0 and xwords.cells[yi+dy-1][xi].isBlocker() == False: dy -= 1
                down_clue_num   = xwords.cells[yi+dy][xi].__cluenum__
                down_clue       = xwords.clue(down_clue_num, 'down')
                down_clue_len   = xwords.numberOfLetters(down_clue_num, 'down')
                down_tuple      = (down_clue_num, 'down')
                dx = 0
                while xi+dx > 0 and xwords.cells[yi][xi+dx-1].isBlocker() == False: dx -= 1
                across_clue_num = xwords.cells[yi][xi+dx].__cluenum__
                across_clue     = xwords.clue(across_clue_num, 'across')
                across_clue_len = xwords.numberOfLetters(across_clue_num, 'across')
                across_tuple    = (across_clue_num, 'across')
                _prompt_        = createPrompt(down_clue, down_clue_len, -dy, across_clue, across_clue_len, -dx)
                down_answer, across_answer = promptModel(_prompt_)
                down_answer   = cleanupAnswer(down_answer)
                across_answer = cleanupAnswer(across_answer)
                if len(down_answer) == down_clue_len and len(across_answer) == across_clue_len:
                    print(f'"{down_clue}" == {down_answer} ### "{across_clue}" == {across_answer}')
                    updateAnswers(down_tuple,   down_answer)
                    updateAnswers(across_tuple, across_answer)
                    down_lu[(xi,yi)]   = down_answer
                    across_lu[(xi,yi)] = across_answer
                elif len(down_answer) != down_clue_len and len(across_answer) == across_clue_len:
                    print(f'-"{across_clue}" == {across_answer}')
                    updateAnswers(across_tuple, across_answer)
                    across_lu[(xi,yi)] = across_answer
                elif len(down_answer) == down_clue_len and len(across_answer) != across_clue_len:
                    print(f'|"{down_clue}" == {down_answer}')
                    updateAnswers(down_tuple,   down_answer)
                    down_lu[(xi,yi)]   = down_answer
                else:
                    print('!',end='')
    saveResults()
else:
    df_cross, df_per_cell = pd.read_parquet(_cross_file_), pd.read_parquet(_per_cell_file_)
    for i in range(len(df_cross)):
        _tuple_ = (df_cross['clue_num'][i], df_cross['orientation'][i])
        _answer_ = df_cross['answer'][i]
        _count_  = df_cross['count'][i]
        if _tuple_ not in _answers_.keys(): _answers_[_tuple_] = {}
        _answers_[_tuple_][_answer_] = _count_
    for i in range(len(df_per_cell)):
        _tuple_       = (df_per_cell['xi'][i], df_per_cell['yi'][i])
        _orientation_ = df_per_cell['orientation'][i]
        _answer_      = df_per_cell['answer'][i]
        if   _orientation_ == 'down':   down_lu[_tuple_]   = _answer_
        elif _orientation_ == 'across': across_lu[_tuple_] = _answer_
        else: raise Exception(f'unknown orientation {_orientation_}')


In [ ]:
xwords.clearAll()
for _tuple_ in _answers_.keys():
    if len(_answers_[_tuple_]) == 1:
        _answer_ = list(_answers_[_tuple_].keys())[0]
        xwords.guess(_tuple_[0], _tuple_[1], _answer_)
print(xwords.characterLevelAccuracy())
xwords

In [ ]:
xwords.clearAll()
_corrects_, _incorrects_ = 0, 0
for _tuple_ in _answers_.keys():
    _clue_num_, _orientation_ = _tuple_
    _answer_actual_ = xwords.answer(_clue_num_, _orientation_)
    _found_ = False
    for _answer_ in _answers_[_tuple_].keys():
        if _answer_actual_.lower() == _answer_.lower(): 
            _found_ = True
            xwords.guess(_tuple_[0], _tuple_[1], _answer_)
    if _found_: _corrects_   += 1
    else:       _incorrects_ += 1
print(f'corrects: {_corrects_}, incorrects: {_incorrects_} [best possible]')
xwords